In [323]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important }</style>"))

# Data Structures

Here we define a node object, used to create a graph for our search algorithms to traverse. We also define 3 priority queues, each having very similar functionaility.

- UCS Priority Queue sorts by g(n) (actual cost)
- GBFS Priority Queue sorts by h(n) (heuristic cost)
- A* Priority Queue sorts by f(n) (g(n) + h(n))

In [324]:
# Define a node class, which is represented by a puzzle string (which corresponds to the state) a parent node, a cost and a heuristic cost.
class Node:
    def __init__(self, puzzle, parent = None, cost = 0, heuristic_cost = 0, move = None):
        self.puzzle = puzzle
        self.parent = parent
        self.cost = cost
        self.heuristic_cost = heuristic_cost
        self.total_cost = cost + heuristic_cost
        self.move = move
    
    # Define a function that returns the a list of moves from the current node to the root node.
    def solution(self):
        path = []
        node = self
        while node.move != None:
            moves.append(node.move)
            node = node.parent
        return path[::-1]
    
class PriorityQueueUCS:
    def __init__(self):
        self.queue = []
    
    def push(self, node):
        self.queue.append(node)
    
    def pop(self):
        try:
            min_val = 0
            for i in range(len(self.queue)):
                if self.queue[i].cost < self.queue[min_val].cost:
                    min_val = i
            item = self.queue[min_val]
            del self.queue[min_val]
            return item
        except IndexError:
            print()
 
    def is_empty(self):
        return len(self.queue) == 0

    def contains(self, state):
        for node in self.queue:
            if node.puzzle[:36] == state:
                return True
        return False

    def get(self, state):
        for node in self.queue:
            if node.puzzle[:36] == state:
                return node
        return None

    def remove(self, node):
        self.queue.remove(node)
    
class PriorityQueueGBFS:
    def __init__(self):
        self.queue = []
    
    def push(self, node):
        self.queue.append(node)
    
    def pop(self):
        try:
            min_val = 0
            for i in range(len(self.queue)):
                if self.queue[i].heuristic_cost < self.queue[min_val].heuristic_cost:
                    min_val = i
            item = self.queue[min_val]
            del self.queue[min_val]
            return item
        except IndexError:
            print()
 
    def is_empty(self):
        return len(self.queue) == 0

    def contains(self, state):
        for node in self.queue:
            if node.puzzle[:36] == state:
                return True
        return False

    def get(self, state):
        for node in self.queue:
            if node.puzzle[:36] == state:
                return node
        return None

    def remove(self, node):
        self.queue.remove(node)

class PriorityQueueAStar:
    def __init__(self):
        self.queue = []
    
    def push(self, node):
        self.queue.append(node)
    
    def pop(self):
        try:
            min_val = 0
            for i in range(len(self.queue)):
                if (self.queue[i].heuristic_cost + self.queue[i].cost) < (self.queue[min_val].heuristic_cost + self.queue[min_val].cost):
                    min_val = i
            item = self.queue[min_val]
            del self.queue[min_val]
            return item
        except IndexError:
            print()
 
    def is_empty(self):
        return len(self.queue) == 0

    def contains(self, state):
        for node in self.queue:
            if node.puzzle[:36] == state:
                return True
        return False

    def get(self, state):
        for node in self.queue:
            if node.puzzle[:36] == state:
                return node
        return None

    def remove(self, state):
        for node in self.queue:
            if node.puzzle[:36] == state:
                self.queue.remove(node)

# Game Logic

Here we define the game logic for the game 'RUSH HOUR'. We implemented the following functionality:

- **get_moves()** takes a puzzle string as input and returns a list of possible moves given that state. Each move/tuple has the form ('CAR', 'DIRECTION', 'DISTANCE'). This method was used to generate the children nodes of a given state.

- **is_goal()** takes in a puzzle string, and checks if the ambulance has reached the exit (if the state is the goal state), indicating the end of the game.

- **remove_car()** takes in a puzzle string, and removes any horizontal car with a position at the exit. This is checked before every move to ensure the generation of children is correct.

- **move()** takes in a puzzle string and a move tuple. It changes the puzzle string based on the move, and returns the new modified board.

In [325]:
import numpy as np

# A puzzle string looks like: 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0 B4'. The first 36 characters represent the puzzle, and the following characters represent the fuel level of the car. In this example, car B has 4 fuel left. 

# Each car has positions represented as indexes. The first column corresponds to all positions index mod 6 = 0, the second column corresponds to all positions index mod 6 = 1, and so on. The first row corresponds to all positions index < 6, the second row corresponds to all positions 6 <= index < 12, and so on. A car has more than 2 positions, and the orientation is determined if the car has more than 1 position in the same row or column. For example if car B has positions 0, 6, 12, 18, 24, 30, then the orientation is vertical, and if car B has positions 0, 1, 2, 3, 4, 5, then the orientation is horizontal. 

# A car can only move in the direction of its orientation. For example, if car B has positions 6, 12, 18, 24, then it can only move up or down. If car B has positions 1, 2, 3, 4, then it can only move left or right. 

# A car cannot move if it is blocked by another car. For example, if car B has positions 6, 12, 18, 24, and car C has positions 0, 1, 2, then car B cannot move up, because car C is blocking it.

# A car cannot move if it is out of bounds. Any position greater than 35 is out of bounds.

# A car cannot move if it has no fuel. For example, if car B has positions 6, 12, 18, 24, and the fuel level of car B is 0, then car B cannot move.

# When a car is moved, all the positions of the car are moved in the direction of the orientation. For example, if car B has positions 6, 12, 18, 24, and the fuel level of car B is 2, then car B can move up or down, and if it moves up, then the positions of car B will be 0, 6, 12, 18.

# If a car is moved, then the fuel level of that car is reduced by the number of positions it moved. For example, if car B has positions 0, 1, 2, and we move it to positions 3, 4, 5, then the fuel level of car B is reduced by 3.

# Define a function that takes a puzzle string and returns a list of all possible moves. Each move is represented by a tuple (car, direction, distance), where car is the car that is moved, direction is the direction that the car is moved, and distance is the number of positions moved. For example, for the puzzle 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0 B4', we should return [('B', 'right', 1), ('B', 'right', 2), ('D', 'right', 1), ('G', 'left', 1), ('G', 'right', 1), ('G', 'right', 2)]
def get_moves(puzzle):
    puzzle_array = np.array(list(puzzle[:36]))
    puzzle_array = remove_car(puzzle_array)
    moves = []
    for i in range(36):
        fuel = 100
        if puzzle_array[i] != '.':
            car = puzzle[i]
            for f in puzzle[37:].split():
                if f[0] == car:
                    fuel = int(f[1:])
            if fuel > 0:
                # If car orientation is horizontal, then it can only move left or right.
                if (i + 1) % 6 != 0 and puzzle[i + 1] == car:
                    # check if index is in first column, and if there is free space on the left
                    # If car can move left, then add move to list.
                    if i % 6 != 0 and puzzle[i - 1] == '.':
                        moves.append((car, 'left', 1))
                        # check if index is in second column, and if there is free space on the left two blocks away
                        # If car can move left twice
                        if(i - 1) % 6 != 0 and puzzle[i - 2] == '.' and fuel > 1:
                            moves.append((car, 'left', 2))
                            # check if index is in third column,and if there is free space on the left three blocks away
                            # If car can move left thrice, then add move to list.
                            if (i - 2) % 6 != 0 and puzzle[i - 3] == '.' and fuel > 2:
                                moves.append((car, 'left', 3))
                                # check if index is in fourth column,and if there is free space on the left four blocks away
                                # If car can move left four times, then add move to list.
                                if (i - 3) % 6 != 0 and puzzle[i - 4] == '.' and fuel > 3:
                                    moves.append((car, 'left', 4))
                    # If car can move right, then add move to list.
                    # check if index is in fifth column, and if there is free space on the right
                    if (i + 2) % 6 != 0 and puzzle[i + 2] == '.':
                        moves.append((car, 'right', 1))
                        # If car can move right twice, then add move to list.
                        # check if index is in fourth column, and if there is free space on the right two blocks away
                        if (i + 3) % 6 != 0 and puzzle[i + 3] == '.' and fuel > 1:
                            moves.append((car, 'right', 2))
                            # If car can move right thrice, then add move to list.
                            # check if index is in third column, and if there is free space on the right three blocks away
                            if (i + 4) % 6 != 0 and puzzle[i + 4] == '.' and fuel > 2:
                                moves.append((car, 'right', 3))
                                # If car can move right four times, then add move to list.
                                # check if index is in second column, and if there is free space on the right four blocks away
                                if (i + 5) % 6 != 0 and puzzle[i + 5] == '.' and fuel > 3:
                                    moves.append((car, 'right', 4))
                # If car orientation is vertical, then it can only move up or down.
                if i + 6 < 36 and puzzle[i + 6] == car:
                    # If car can move up, then add move to list.
                    if i - 6 >= 0 and puzzle[i - 6] == '.':
                        moves.append((car, 'up', 1))
                        # If car can move up twice, then add move to list.
                        if i - 12 >= 0 and puzzle[i - 12] == '.' and fuel > 1:
                            moves.append((car, 'up', 2))
                            # If a car can move up 3 times, then add move to list.
                            if i - 18 >= 0 and puzzle[i - 18] == '.' and fuel > 2:
                                moves.append((car, 'up', 3))
                                # If a car can move up 4 times, then add move to list.
                                if i - 24 >= 0 and  puzzle[i - 24] == '.' and fuel > 3:
                                    moves.append((car, 'up', 4))
                    # If car can move down, then add move to list.
                    if i + 12 < 36 and puzzle[i + 12] == '.':
                        moves.append((car, 'down', 1))
                        # If car can move down twice, then add move to list.
                        if i + 18 < 36 and puzzle[i + 18] == '.' and fuel > 1:
                            moves.append((car, 'down', 2))
                            # If a car can move down 3 times, then add move to list.
                            if i + 24 < 36 and puzzle[i + 24] == '.' and fuel > 2:
                                moves.append((car, 'down', 3))
                                # If a car can move down 4 times, then add move to list.
                                if i + 30 < 36 and puzzle[i + 30] == '.' and fuel > 3:
                                    moves.append((car, 'down', 4))
                
    return moves

# Define a function that checks if index 17 is occupied by the A car, return True if it is, False otherwise.
def is_goal(puzzle):
    puzzle_array = np.array(list(puzzle[:36]))
    return puzzle_array[17] == 'A'

# Define a function that first checks the 17th index of the puzzle, if it is occupied by any car other than the A car, then continue. If the 16th index is occupied by the same car as the 17th index, then continue. Repeat this process until the car is no longer the same as the car in the 17th index. Once the process is complete, replace all instances of the car in the 17th index with a period. 
def remove_car(puzzle):
    car = puzzle[17]
    if car == '.':
        return puzzle
    elif car == puzzle[16] and car != 'A':
        for i in range(17, 0, -1):
            if puzzle[i] != car:
                break
            puzzle[i] = '.'
    return puzzle
 
# Define a function that takes a puzzle string and a move, and returns the puzzle string after the move is applied. For example, if the puzzle string is 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0' and the move is ('B', 'right', 2), then the function returns '..BBBMCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII J0 B98'. If the car fuel is not defined, then the fuel level of the car is 100. Move all indexes of the car in the direction specified by the move. 
def move(puzzle, move):
    puzzle_array = np.array(list(puzzle[:36]))
    fuel_string = puzzle[37:].split()
    car = move[0]
    direction = move[1]
    distance = move[2]
    fuel = 100
    for f in puzzle[37:].split():
        if f[0] == car:
            fuel = int(f[1:])
    if direction == 'left':
        for i in range(36):
            if puzzle_array[i] == car:
                puzzle_array[i] = '.'
                puzzle_array[i - distance] = car
    elif direction == 'right':
        for i in range(35, -1, -1):
            if puzzle_array[i] == car:
                puzzle_array[i] = '.'
                puzzle_array[i + distance] = car
    elif direction == 'up':
        for i in range(36):
            if puzzle_array[i] == car:
                puzzle_array[i] = '.'
                puzzle_array[i - 6 * distance] = car
    elif direction == 'down':
        for i in range(35, -1, -1):
            if puzzle_array[i] == car:
                puzzle_array[i] = '.'
                puzzle_array[i + 6 * distance] = car
    puzzle_array = remove_car(puzzle_array)
    new_puzzle = ''.join(puzzle_array)
    # Update the fuel level of the car, and concatenate the original fuel string with the new fuel string. If the original fuel string looks like 'F0 B4', and we move the B car 2 spaces to the right, then the new fuel string will be 'F0 B2'. If the fuel string is empty and we apply the same move, then the new fuel string will be 'B2'.
    if fuel_string:
        for i in range(len(fuel_string)):
            if fuel_string[i][0] == car:
                fuel_string[i] = car + str(fuel - distance)
                break
            elif i == len(fuel_string) - 1:
                fuel_string.append(car + str(fuel - distance))
    else:
        fuel_string.append(car + str(fuel - distance))
    new_puzzle += ' ' + ' '.join(fuel_string)
    return new_puzzle

def print_puzzle(puzzle):
    for i in range(36):
        if i %6== 0 and i >0:
            print("")
        print(puzzle[i], " ", sep="", end="")
    

# Heuristics

As we are using two informed searches (namely A/A* and GBFS), the algorithms make use of heuristics functions. Here we define 4 unique heuristics, please read comments for further description.

In [326]:
# Define each heuristic as a function, which takes in a puzzle string and returns an integer based on its rules.

# Heuristic 1: The number of unique cars in between the A car and the 17th index. For example the puzzle 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII' has 3 unique cars in between the A car and the 17th index, namely K, L and M.
def h1(puzzle):
    puzzle_array = np.array(list(puzzle[:36]))
    # Get the last index of the A car.
    index = 0
    for i in range(35, -1, -1):
        if puzzle_array[i] == 'A':
            index = i
            break
    # Get the number of unique cars in between the i'th and 17th index.
    unique_cars = []
    for i in range(index + 1, 18):
        if puzzle_array[i] not in unique_cars and puzzle_array[i] != '.':
            unique_cars.append(puzzle_array[i])
    return len(unique_cars)

# Heuristic 2: the number of blocked positions in between the A car and the 17th index. For example the puzzle 'BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII' has 3 blocked positions in between the A car and the 17th index, namely the 14th, 15th and 17th index.
def h2(puzzle):
    puzzle_array = np.array(list(puzzle[:36]))
    # Get the last index of the A car.
    index = 0
    for i in range(35, -1, -1):
        if puzzle_array[i] == 'A':
            index = i
            break
    # Get the number of blocked positions in between the i'th and 17th index.
    blocked_positions = 0
    for i in range(index + 1, 18):
        if puzzle_array[i] != '.':
            blocked_positions += 1
    return blocked_positions

# Heuristic 3: Heuristic 1 * constant parameter lambda (defaulted to 3).
def h3(puzzle, lam = 3):
    return h1(puzzle) * lam

# Heuristic 4: Heuristic 2 * 1/3.
def h4(puzzle):
    return h2(puzzle) / 3

# Algorithms

Here we define our search algorithms. The first being **Uniform Cost Search**, an uninformed search which gaurantees an optimal solution. The second being **Greedy Best First Search**, an informed search which does not necessarily gaurantee an optimal solution, but will return a solution if one exists. Lastly we define **A/A* Algorithm**, which is another informed search algorithm, which does gaurantee an optimal solution if the heuristic being used is admissable.  

In [327]:
import time

def ucs(puzzle, num):
    filename = 'ucs-search-' + str(num) + '.txt' 
    initial_puzzle = puzzle
    
    tic = time.perf_counter()

    root = Node(puzzle, parent = None, cost = 0, heuristic_cost = 0, move = None)

    open = PriorityQueueUCS()
    closed = set()

    open.push(root)

    pop_counter = 0
    while True:
        try:
            node = open.pop()
            print_path(filename, node)
            pop_counter +=1
            if node is None:
                raise AttributeError("")
        except IndexError:
            toc = time.perf_counter()
            print(f"No solution found in {toc - tic:0.4f} seconds")
            print("")
            filename = 'ucs-sol-' + str(num) + '.txt'
            print_solution(filename, initial_puzzle, toc - tic, pop_counter)
            analysis(num, 'UCS ', 'N/A', 'N/A', pop_counter, toc - tic)
            return 'No Solution.'
        except AttributeError:
            toc = time.perf_counter()
            print(f"No solution found in {toc - tic:0.4f} seconds")
            print("")
            filename = 'ucs-sol-' + str(num) + '.txt'
            print_solution(filename, initial_puzzle, toc - tic, pop_counter)
            analysis(num, 'UCS ', 'N/A', 'N/A', pop_counter, toc - tic)
            return 'No Solution.'

        if is_goal(node.puzzle):
            toc = time.perf_counter()
            print("Solution Found.")
            print(f"Time elapsed: {toc - tic:0.4f} seconds")
            print("")
            solution = []
            solution_node = node
            while node.parent is not None:
                solution.append(node.move)
                node = node.parent
            solution.reverse()
            filename = 'ucs-sol-' + str(num) + '.txt'
            print_solution(filename, initial_puzzle, toc - tic, pop_counter, solution, True, solution_node.puzzle, solution_node)
            analysis(num, 'UCS ', 'N/A', len(solution), pop_counter, toc - tic)
            return 'Solution Found'
        else:
            children = set()
            for moves in get_moves(node.puzzle):
                puzzle = move(node.puzzle, moves)
                children.add(Node(puzzle, parent = node, cost = node.cost + 1, heuristic_cost = 0, move = moves))

            closed.add(node.puzzle[:36])

            for child in children:
                if child.puzzle[:36] not in closed:
                    if open.contains(child.puzzle[:36]) == False:
                        open.push(child)
                    else:
                        if child.heuristic_cost < open.get(child.puzzle[:36]).heuristic_cost:
                            open.remove(child)
                            open.push(child)


In [328]:
import time

def gbfs(puzzle, heuristic, num):
    filename = 'gbfs-' + heuristic.__name__ + '-search-' + str(num) + '.txt'
    initial_puzzle = puzzle
    
    tic = time.perf_counter()

    root = Node(puzzle, parent = None, cost = 0, heuristic_cost = heuristic(puzzle), move = None)

    open = PriorityQueueGBFS()
    closed = set()

    open.push(root)

    pop_counter = 0
    while True:
        try:
            node = open.pop()
            print_path(filename, node)
            pop_counter +=1
            if node is None:
                raise AttributeError("")
        except IndexError:
            toc = time.perf_counter()
            print(f"No solution found in {toc - tic:0.4f} seconds")
            print("")
            filename = 'gbfs-' + heuristic.__name__ + '-sol-' + str(num) + '.txt'
            print_solution(filename, initial_puzzle, toc - tic, pop_counter)
            analysis(num, 'GBFS', heuristic.__name__, 'N/A', pop_counter, toc - tic)
            return 'No Solution.'
        except AttributeError:
            toc = time.perf_counter()
            print(f"No solution found in {toc - tic:0.4f} seconds")
            print("")
            filename = 'gbfs-' + heuristic.__name__ + '-sol-' + str(num) + '.txt'
            print_solution(filename, initial_puzzle, toc - tic, pop_counter)
            analysis(num, 'GBFS', heuristic.__name__, 'N/A', pop_counter, toc - tic)
            return 'No Solution.'

        if is_goal(node.puzzle):
            toc = time.perf_counter()
            print("Solution Found.")
            print(f"Time elapsed: {toc - tic:0.4f} seconds")
            print("")
            solution = []
            solution_node = node
            while node.parent is not None:
                solution.append(node.move)
                node = node.parent
            filename = 'gbfs-' + heuristic.__name__ + '-sol-' + str(num) + '.txt'
            solution.reverse()
            print_solution(filename, initial_puzzle, toc - tic, pop_counter, solution, True, solution_node.puzzle, solution_node)
            analysis(num, 'GBFS', heuristic.__name__, len(solution), pop_counter, toc - tic)
            return 'Solution Found'
        else:
            children = set()
            for moves in get_moves(node.puzzle):
                puzzle = move(node.puzzle, moves)
                children.add(Node(puzzle, parent = node, cost = node.cost + 1, heuristic_cost = heuristic(puzzle), move = moves))

            closed.add(node.puzzle[:36])

            for child in children:
                if child.puzzle[:36] not in closed:
                    if open.contains(child.puzzle[:36]) == False:
                        open.push(child)
                    else:
                        if child.heuristic_cost < open.get(child.puzzle[:36]).heuristic_cost:
                            open.remove(child)
                            open.push(child)


In [329]:
import time

def astar(puzzle, heuristic, num):
    filename = 'a-' + heuristic.__name__ + '-search-' + str(num) + '.txt'
    initial_puzzle = puzzle
    
    tic = time.perf_counter()

    root = Node(puzzle, parent = None, cost = 0, heuristic_cost = heuristic(puzzle), move = None)

    open = PriorityQueueAStar()
    closed = PriorityQueueAStar()

    open.push(root)

    pop_counter = 0
    while True:
        try:
            node = open.pop()
            print_path(filename, node)
            pop_counter +=1
            if node is None:
                raise AttributeError("")
        except IndexError:
            toc = time.perf_counter()
            print(f"No solution found in {toc - tic:0.4f} seconds")
            print("")
            filename = 'a-' + heuristic.__name__ + '-sol-' + str(num) + '.txt'
            print_solution(filename, initial_puzzle, toc - tic, pop_counter)
            analysis(num, 'A/A*', heuristic.__name__, 'N/A', pop_counter, toc - tic)
            return 'No Solution.'
        except AttributeError:
            toc = time.perf_counter()
            print(f"No solution found in {toc - tic:0.4f} seconds")
            print("")
            filename = 'a-' + heuristic.__name__ + '-sol-' + str(num) + '.txt'
            print_solution(filename, initial_puzzle, toc - tic, pop_counter)
            analysis(num, 'A/A*', heuristic.__name__, 'N/A', pop_counter, toc - tic)
            return 'No Solution.'

        if is_goal(node.puzzle):
            toc = time.perf_counter()
            print("Solution Found.")
            print(f"Time elapsed: {toc - tic:0.4f} seconds")
            print("")
            solution = []
            solution_node = node
            while node.parent is not None:
                solution.append(node.move)
                node = node.parent
            filename = 'a-' + heuristic.__name__ + '-sol-' + str(num) + '.txt'
            solution.reverse()
            print_solution(filename, initial_puzzle, toc - tic, pop_counter, solution, True, solution_node.puzzle, solution_node)
            analysis(num, 'A/A*', heuristic.__name__, len(solution), pop_counter, toc - tic)
            return 'Solution Found'
        else:
            children = set()
            for moves in get_moves(node.puzzle):
                puzzle = move(node.puzzle, moves)
                children.add(Node(puzzle, parent = node, cost = node.cost + 1, heuristic_cost = heuristic(puzzle), move = moves))

            closed.push(node)

            for child in children:
                if closed.contains(child.puzzle[:36]) == False:
                    if open.contains(child.puzzle[:36]) == False:
                        open.push(child)
                    else:
                        if (child.cost + child.heuristic_cost) < (open.get(child.puzzle[:36]).cost + open.get(child.puzzle[:36]).heuristic_cost):
                            open.remove(child.puzzle)
                            open.push(child)
                else:
                    if (child.cost + child.heuristic_cost) < (closed.get(child.puzzle[:36]).cost + closed.get(child.puzzle[:36]).heuristic_cost):
                        closed.remove(child)
                        open.push(child)

# I/O

The last two cells defined are used to handle file reading, puzzle parsing and printing the solutions and search paths to files outside of the notebook. 

In [330]:
# Function that takes file name as input, and returns a list of puzzles. Each line in the file is a puzzle. If a line is empty, it is ignored. If a line starts with #, it is ignored.
def read_puzzles(file_name):
    with open(file_name, 'r') as file:
        for line in file:
            if line.startswith('#') or line == ' ':
                continue
            else:
                puzzles.append(line.strip())
    # Remove empty lines from the list.
    puzzles[:] = [puzzle for puzzle in puzzles if puzzle != '']
    return puzzles

# The first digit represents the (cost + heuristic cost) of the node at that point in time [f(n)]. The second digit represents the cost [g(n)] of the node at that point in time. The third digit represents the heuristic cost [h(n)] of the node at that point in time. Even if no solution was found, the entire search path is printed in the file. 
def print_path(file_name, node):
    heuristic_cost = "%.2f" % node.heuristic_cost
    cost = "%.2f" % node.cost
    total_cost = "%.2f" % (node.cost + node.heuristic_cost)
    with open("Output/Search/"+file_name, 'a') as file:
        file.write(total_cost + ' ' + cost + ' ' + heuristic_cost + ' ' + node.puzzle + ' \n')

def print_solution(file_name, initial_puzzle, time, search_length, solution = None, solution_found = False, solved_puzzle = None, node=None):
    with open("Output/Solution/"+file_name, 'w') as file:
        file.write('----------------------------------------------------------------------------\n')
        file.write('\n')
        file.write('Initial Board Configuration: ' + initial_puzzle + '\n')
        file.write('\n')
        # Print the puzzle as a 6x6 grid.
        file.write(' '.join(initial_puzzle[0:6]) + '\n')
        file.write(' '.join(initial_puzzle[6:12]) + '\n')
        file.write(' '.join(initial_puzzle[12:18]) + '\n')
        file.write(' '.join(initial_puzzle[18:24]) + '\n')
        file.write(' '.join(initial_puzzle[24:30]) + '\n')
        file.write(' '.join(initial_puzzle[30:36]) + '\n')
        file.write('\n')
        
        # Get all unique cars in the puzzle
        cars = set()
        for i in range(len(initial_puzzle[:36])):
            if initial_puzzle[i] != '.':
                cars.add(initial_puzzle[i])
        file.write('Car Fuel Available: ')
        fuel_string = initial_puzzle[37:] + ' '
        # Print each car and its fuel available. If a car does not have a defined fuel level, it is assumed to have a fuel level of 100.
        for car in cars:
            if car in fuel_string:
                file.write(car + ':' + fuel_string[fuel_string.index(car) + 1] + fuel_string[fuel_string.index(car) + 2] + ' ')
            else:
                file.write(car + ':100 ')
                    
        file.write('\n')
        file.write('\n')
        file.write('Runtime: ' + str(time) + ' seconds \n')
        file.write('\n')
        file.write('Search Path Length: ' + str(search_length) + ' states \n')
        file.write('\n')

        if solution_found == False:
            file.write('Solution: No Solution \n')
            file.write('\n')
            file.write('----------------------------------------------------------------------------')
        else:
            file.write('Solution Path Length: ' + str(len(solution)) + ' moves \n')
            file.write('\n')
            # Print the list of moves in the solution.
            file.write('Solution Path: ')
            for i in range(len(solution)):
                move = ' '.join(map(str, solution[i]))
                file.write(move + ', ')
            file.write('\n')
            file.write('\n')

            nodes = []
            while node.parent is not None:
                nodes.append(node)
                node = node.parent
            nodes.reverse()
            for node in nodes:
                file.write(str(node.move[0]) + '\t' + str(node.move[1]) + '\t' + str(node.move[2]) + '\t' + node.puzzle + '\n')

            file.write('\n')
            file.write(' '.join(solved_puzzle[0:6]) + '\n')
            file.write(' '.join(solved_puzzle[6:12]) + '\n')
            file.write(' '.join(solved_puzzle[12:18]) + '\n')
            file.write(' '.join(solved_puzzle[18:24]) + '\n')
            file.write(' '.join(solved_puzzle[24:30]) + '\n')
            file.write(' '.join(solved_puzzle[30:36]) + '\n')
            file.write('\n')
            # Get all unique cars in the puzzle
            cars = set()
            for i in range(len(solved_puzzle[:36])):
                if solved_puzzle[i] != '.':
                    cars.add(solved_puzzle[i])
            file.write('Car Fuel Remaining: ')
            fuel_string = solved_puzzle[37:] + ' '
            # Print each car and its fuel available. If a car does not have a defined fuel level, it is assumed to have a fuel level of 100.
            for car in cars:
                if car in fuel_string:
                    file.write(car + ':' + fuel_string[fuel_string.index(car) + 1] + fuel_string[fuel_string.index(car) + 2] + ' ')
                else:
                    file.write(car + ':100 ')
            file.write('\n')
            file.write('\n')
            file.write('----------------------------------------------------------------------------')


def analysis(puzzle_num, algo_name, heuristic_name, sol_length, search_length, time):
    time = "%.4f" % time
    with open("Analysis.csv", 'a') as file:
        file.write(str(puzzle_num ) + ',' + algo_name + ',' + heuristic_name + ',' + str(sol_length) + ',' + str(search_length) + ',' + time + '\n')


In [333]:
# Open the file and read the puzzles.
puzzles = []
puzzles = read_puzzles('Input.txt')

# Run the search algorithm on each puzzle.
for i in range(len(puzzles)):
    gbfs(puzzles[i], h1, i+1)
    gbfs(puzzles[i], h2, i+1)
    gbfs(puzzles[i], h3, i+1)
    gbfs(puzzles[i], h4, i+1)
    astar(puzzles[i], h1, i+1)
    astar(puzzles[i], h2, i+1)
    astar(puzzles[i], h3, i+1)
    astar(puzzles[i], h4, i+1)
    ucs(puzzles[i], i+1)
    

Solution Found.
Time elapsed: 0.2065 seconds

Solution Found.
Time elapsed: 0.1565 seconds

Solution Found.
Time elapsed: 0.1573 seconds

Solution Found.
Time elapsed: 0.1427 seconds

Solution Found.
Time elapsed: 2.4562 seconds

Solution Found.
Time elapsed: 2.4038 seconds

Solution Found.
Time elapsed: 0.2407 seconds

Solution Found.
Time elapsed: 3.5001 seconds

Solution Found.
Time elapsed: 0.9847 seconds

Solution Found.
Time elapsed: 0.0056 seconds

Solution Found.
Time elapsed: 0.0055 seconds

Solution Found.
Time elapsed: 0.0063 seconds

Solution Found.
Time elapsed: 0.0062 seconds

Solution Found.
Time elapsed: 0.0089 seconds

Solution Found.
Time elapsed: 0.0088 seconds

Solution Found.
Time elapsed: 0.0061 seconds

Solution Found.
Time elapsed: 0.0089 seconds

Solution Found.
Time elapsed: 0.0066 seconds

Solution Found.
Time elapsed: 0.0039 seconds

Solution Found.
Time elapsed: 0.0046 seconds

Solution Found.
Time elapsed: 0.0047 seconds

Solution Found.
Time elapsed: 0.00

Solution Found.
Time elapsed: 0.1032 seconds

Solution Found.
Time elapsed: 0.0297 seconds

Solution Found.
Time elapsed: 0.0301 seconds

Solution Found.
Time elapsed: 0.0315 seconds

Solution Found.
Time elapsed: 0.0308 seconds

Solution Found.
Time elapsed: 0.2042 seconds

Solution Found.
Time elapsed: 0.1980 seconds

Solution Found.
Time elapsed: 0.0803 seconds

Solution Found.
Time elapsed: 0.2104 seconds

Solution Found.
Time elapsed: 0.1149 seconds

Solution Found.
Time elapsed: 0.0114 seconds

Solution Found.
Time elapsed: 0.0123 seconds

Solution Found.
Time elapsed: 0.0142 seconds

Solution Found.
Time elapsed: 0.0106 seconds

Solution Found.
Time elapsed: 0.0348 seconds

Solution Found.
Time elapsed: 0.0352 seconds

Solution Found.
Time elapsed: 0.0130 seconds

Solution Found.
Time elapsed: 0.0366 seconds

Solution Found.
Time elapsed: 0.0382 seconds

Solution Found.
Time elapsed: 0.0513 seconds

Solution Found.
Time elapsed: 0.0580 seconds

Solution Found.
Time elapsed: 0.06

Solution Found.
Time elapsed: 0.3495 seconds

Solution Found.
Time elapsed: 0.3189 seconds

Solution Found.
Time elapsed: 0.0248 seconds

Solution Found.
Time elapsed: 0.0315 seconds

Solution Found.
Time elapsed: 0.0201 seconds

Solution Found.
Time elapsed: 0.0288 seconds

Solution Found.
Time elapsed: 0.7149 seconds

Solution Found.
Time elapsed: 0.7010 seconds

Solution Found.
Time elapsed: 0.0588 seconds

Solution Found.
Time elapsed: 1.1794 seconds

Solution Found.
Time elapsed: 0.5975 seconds

Solution Found.
Time elapsed: 0.0781 seconds

Solution Found.
Time elapsed: 0.0611 seconds

Solution Found.
Time elapsed: 0.0573 seconds

Solution Found.
Time elapsed: 0.0532 seconds

Solution Found.
Time elapsed: 1.3071 seconds

Solution Found.
Time elapsed: 1.2888 seconds

Solution Found.
Time elapsed: 0.3801 seconds

Solution Found.
Time elapsed: 1.9899 seconds

Solution Found.
Time elapsed: 0.6523 seconds

Solution Found.
Time elapsed: 0.0647 seconds

Solution Found.
Time elapsed: 0.04